# Naas Storage

## Input

### Import libraries

In [ ]:
import naas_python
import pydash
import naas_data_product

### Setup variables

In [ ]:
# Naas API
workspace_id = None
storage_name = "abi"

# Boto3
access_key_id = None
secret_access_key = None
session_token = None
bucket_name = None
bucket_prefix = None

## Model

### Get credentials

In [ ]:
def get_storage_credentials(
    workspace_id=None,
    storage_name=None,
):
    # Init
    if workspace_id is None:
        api_key = naas_python.secret.get('NAAS_API_TOKEN').value
        workspace_id = get_personal_workspace(api_key)
#     print("Workspace ID:", workspace_id)
        
    # List storage
    result = naas_python.storage.list_workspace_storage(workspace_id=workspace_id)
    storages = result.get("storage")
    storage_exist = False
    for storage in storages:
        if storage.get("name") == storage_name:
            storage_exist = True
            new_storage = storage
            
    # Create storage
    if not storage_exist:
        new_storage = naas_python.storage.create_workspace_storage(workspace_id=workspace_id, storage_name=storage_name).get("storage")
        
    # Get storage credentials
    credentials = naas_python.storage.create_workspace_storage_credentials(workspace_id=workspace_id, storage_name=storage_name)
    return credentials

if access_key_id is None and secret_access_key is None:
    credentials = get_storage_credentials(workspace_id=workspace_id, storage_name=storage_name)
    if len(credentials) > 0:
        access_key_id = pydash.get(credentials, "credentials.s3.access_key_id")
        secret_access_key = pydash.get(credentials, "credentials.s3.secret_key")
        session_token = pydash.get(credentials, "credentials.s3.session_token")
        endpoint_url = pydash.get(credentials, "credentials.s3.endpoint_url")
        bucket_name = endpoint_url.split("s3://")[1].split("/")[0]
        bucket_prefix = endpoint_url.split(f"{bucket_name}/")[1]
        region_name = pydash.get(credentials, "credentials.s3.region_name")

### Create Class StorageManager

In [ ]:
import boto3
import pickle
from io import BytesIO
import os

class StorageManager:

    def __init__(
        self,
        access_key_id,
        secret_access_key,
        session_token,
        bucket_name,
        bucket_prefix, 
    ):
        self.access_key_id = access_key_id
        self.secret_access_key = secret_access_key
        self.session_token = session_token
        self.bucket_name = bucket_name
        self.bucket_prefix = bucket_prefix
        
        # Init client
        self.s3 = boto3.client(
            's3', 
            aws_access_key_id=access_key_id, 
            aws_secret_access_key=secret_access_key,
            aws_session_token=session_token,
        )
        
    def __fix_path_prefix(self, path_prefix):
        if "/home/ftp" in path_prefix:
            datalake_dir = naas_python.secret.get("ABI_DATALAKE_DIR").value
            path_prefix = path_prefix.replace(f"{datalake_dir}/", "")
        return path_prefix
        
    def pload(
        self,
        path_prefix,
        file_name
    ):
        # Init path
        file_path = os.path.join(self.bucket_prefix, self.__fix_path_prefix(path_prefix), f'{file_name}.pickle')
        try:
            obj = self.s3.get_object(Bucket=self.bucket_name, Key=file_path)
            bytestream = BytesIO(obj['Body'].read())
            return pickle.load(bytestream)
        except Exception as e:
            print(e)
            return None
        return pickle_data

    def pdump(
        self,
        path_prefix,
        object_to_dump,
        file_name,
    ):
        # Init paths
        file_path = os.path.join(self.bucket_prefix, self.__fix_path_prefix(path_prefix), f'{file_name}.pickle')
        histo_path = os.path.join(self.bucket_prefix, self.__fix_path_prefix(path_prefix), f'{datetime.now().strftime("%Y%m%d%H%M%S")}_{file_name}.pickle')
        
        # Save pickle files
        pickle_byte_obj = BytesIO()
        pickle.dump(object_to_dump, pickle_byte_obj)
        pickle_byte_obj.seek(0)
        self.s3.put_object(Bucket=self.bucket_name, Key=file_path, Body=pickle_byte_obj)
        self.s3.put_object(Bucket=self.bucket_name, Key=histo_path, Body=pickle_byte_obj)
        
    def list_objects(self, path_prefix=None):
        # Init
        files = []
        dir_path = self.bucket_prefix
        if path_prefix is not None:
            dir_path = os.path.join(self.bucket_prefix, self.__fix_path_prefix(path_prefix))
        
        # List objects
        response = self.s3.list_objects_v2(Bucket=self.bucket_name, Prefix=dir_path + "/")
        
        # Process the response
        if 'Contents' in response:
            for file in response['Contents']:
                file_name = file['Key']
                files.append(file_name)
        return files
    
    def delete_object(self, file_path):
        self.s3.delete_object(Bucket=self.bucket_name, Key=file_path)

## Output

### Set Storage Manager

In [ ]:
sm = StorageManager(access_key_id, secret_access_key, session_token, bucket_name, bucket_prefix)